In [4]:
# from tensorflow.keras.utils import img_to_array
from PIL import Image
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from keras.models import Sequential
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
from keras import layers
import random
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from glob import glob
import cv2 as cv
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from keras.models import load_model
from tensorflow.keras.regularizers import l2


In [2]:
import zipfile
import os

zip_path = r"/content/final_leaf.zip"  # Update this with the actual ZIP file path
extract_dir = r"/content/final_leaf.zip\all_data"  # Choose a directory to extract the data

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete!")


Extraction complete!


In [5]:
base_dir = r"/content/final_leaf.zip\all_data/leaf2"

In [6]:
# Image Parameters
img_size = 224
batch_size = 32

In [7]:
# Image Data Generators
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

In [8]:
# Train Generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 4230 images belonging to 15 classes.


In [9]:
# Validation Generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 1050 images belonging to 15 classes.


In [10]:
train_generator.num_classes

15

In [11]:
# Model Definition
from tensorflow.keras import layers, models
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    47,776,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,799,247 (182.34 MB)

 Trainable params: 47,799,247 (182.34 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Number of steps per epoch
    epochs=7,  # Number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size  # Validation steps
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 22s 124ms/step - accuracy: 0.3767 - loss: 3.9820 - val_accuracy: 0.7598 - val_loss: 0.7328
Epoch 2/7
  1/132 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.8750 - loss: 0.4961

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8750 - loss: 0.4961 - val_accuracy: 0.7520 - val_loss: 0.7698
Epoch 3/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 30s 99ms/step - accuracy: 0.8857 - loss: 0.3501 - val_accuracy: 0.8369 - val_loss: 0.5001
Epoch 4/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9688 - loss: 0.1901 - val_accuracy: 0.8262 - val_loss: 0.5174
Epoch 5/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 21s 157ms/step - accuracy: 0.9285 - loss: 0.2219 - val_accuracy: 0.8164 - val_loss: 0.5683
Epoch 6/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9062 - loss: 0.2535 - val_accuracy: 0.8047 - val_loss: 0.6724
Epoch 7/7
132/132 ━━━━━━━━━━━━━━━━━━━━ 21s 158ms/step - accuracy: 0.9705 - loss: 0.1001 - val_accuracy: 0.8516 - val_loss: 0.4340


In [15]:
# Model Evaluation
print("Evaluating model...")
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.8536 - loss: 0.4583
Validation Accuracy: 85.16%


In [16]:
# Function to Load and Preprocess the Image using Pillow
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    img = Image.open(image_path).convert("RGB")
    # Resize the image
    img = img.resize(target_size)
    # Convert the image to a numpy array
    img_array = np.array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Scale the image values to [0, 1]
    img_array = img_array.astype('float32') / 255.
    return img_array

# Function to Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [17]:
# Create a mapping from class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}
train_generator.class_indices.items()
class_indices

{0: 'Alstonia Scholaris',
 1: 'Arjun',
 2: 'Bael',
 3: 'Basil',
 4: 'Chinar',
 5: 'Gauva',
 6: 'Jamun',
 7: 'Lemon',
 8: 'Pomegranate',
 9: 'apple',
 10: 'cotton',
 11: 'grape',
 12: 'mango',
 13: 'potato',
 14: 'tomato'}

In [34]:
im1=r'/content/final_leaf.zip\all_data/leaf2/grape/00a962ad-573b-44b1-97ae-912a6bd6e0b0___FAM_L.Blight 1431.JPG' #grape
im2=r'/content/final_leaf.zip\all_data/leaf2/potato/20230712_114353.jpg'  #potato
im3=r'/content/final_leaf.zip\all_data/leaf2/apple/00fca0da-2db3-481b-b98a-9b67bb7b105c___RS_HL 7708.JPG'  #apple
im4=r'/content/final_leaf.zip\all_data/leaf2/cotton/bact17.jpg'  #cotton
im5=r'/content/final_leaf.zip\all_data/leaf2/mango/20211008_125130 (Custom).jpg'  #mango
im6=r'/content/final_leaf.zip\all_data/leaf2/tomato/0a22f50a-5f25-4cf6-816b-76cae94b7f30___GCREC_Bact.Sp 6103.JPG'  #tomato
im7=r'/content/final_leaf.zip\all_data/leaf2/Alstonia Scholaris/0003_0013.JPG' #AS
im8=r'/content/final_leaf.zip\all_data/leaf2/Arjun/0002_0016.JPG'
im9=r'/content/final_leaf.zip\all_data/leaf2/Bael/0016_0016.JPG'
im10=r'/content/final_leaf.zip\all_data/leaf2/Basil/0008_0017.JPG'
im11=r'/content/final_leaf.zip\all_data/leaf2/Chinar/0011_0022.JPG'
im12=r'/content/final_leaf.zip\all_data/leaf2/Gauva/0004_0017.JPG'
im13=r'//content/final_leaf.zip\all_data/leaf2/Jamun/0005_0017.JPG'
im14=r'/content/final_leaf.zip\all_data/leaf2/Pomegranate/0009_0182.JPG'
im15=r'/content/final_leaf.zip\all_data/leaf2/Lemon/0010_0018.JPG'
predicted_class_name1 = predict_image_class(model, im1, class_indices)
predicted_class_name2 = predict_image_class(model, im2, class_indices)
predicted_class_name3 = predict_image_class(model, im3, class_indices)
predicted_class_name4 = predict_image_class(model, im4, class_indices)
predicted_class_name5 = predict_image_class(model, im5, class_indices)
predicted_class_name6 = predict_image_class(model, im6, class_indices)
predicted_class_name7 = predict_image_class(model, im7, class_indices)
predicted_class_name8 = predict_image_class(model, im8, class_indices)
predicted_class_name9 = predict_image_class(model, im9, class_indices)
predicted_class_name10 = predict_image_class(model, im10, class_indices)
predicted_class_name11 = predict_image_class(model, im11, class_indices)
predicted_class_name12= predict_image_class(model, im12, class_indices)
predicted_class_name13= predict_image_class(model, im13, class_indices)
predicted_class_name14= predict_image_class(model, im14, class_indices)
predicted_class_name15= predict_image_class(model, im15, class_indices)
# print(predicted_class_name6)
# Output the result
print("Predicted Class Name:", predicted_class_name1,predicted_class_name2,predicted_class_name3,predicted_class_name4,predicted_class_name5,
      predicted_class_name6,predicted_class_name7,predicted_class_name8,predicted_class_name9,predicted_class_name10,predicted_class_name11,
      predicted_class_name12,predicted_class_name13,predicted_class_name14,predicted_class_name15)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Class Name: grape potato apple cotton mango tomato Alstonia Scholaris Arjun Bael Basil Chinar Gauva Jamun Pomegranate Lemon


In [ ]:
model.save('final_leaf_prediction_model.h5')

In [ ]:
print(f"Train Samples: {train_generator.samples}")
print(f"Validation Samples: {validation_generator.samples}")
print(f"Batch Size: {batch_size}")

Train Samples: 2105
Validation Samples: 524
Batch Size: 32


In [1]:
from keras.models import load_model

# Load the model using Keras's load_model function
model = load_model(r'C:\Users\Arunava Chakraborty\Desktop\plant-disease-prediction-cnn-deep-leanring-project-main\final_leaf_prediction2_model.h5')

In [2]:
from PIL import Image
import numpy as np

def predict_image(model, image_path, class_indices, target_size=(224, 224)):
    # Load and preprocess the image
    img = Image.open(image_path).convert("RGB")
    img = img.resize(target_size)
    img_array = np.expand_dims(np.array(img).astype('float32') / 255.0, axis=0)
    
    # Make prediction
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    
    return predicted_class_name

In [3]:
class_indices = {
    0: 'Alstonia Scholaris',
    1: 'Arjun',
    2: 'Bael',
    3: 'Basil',
    4: 'Chinar',
    5: 'Gauva',
    6: 'Jamun',
    7: 'Lemon',
    8: 'Pomegranate',
    9: 'apple',
    10: 'cotton',
    11: 'grape',
    12: 'mango',
    13: 'potato',
    14: 'tomato'
}

In [11]:
image_path = r'C:\Users\Arunava Chakraborty\Desktop\plant-disease-prediction-cnn-deep-leanring-project-main\all_data\leaf2\Pomegranate\0009_0081.JPG'
result = predict_image(model, image_path, class_indices)
print("Predicted Class:", result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted Class: Pomegranate
